# Data Ingestion

In [0]:
from datetime import datetime
proj_df = spark.read.csv("/Volumes/workspace/hrd/rawfiles/Project.csv", header=True, inferSchema=True)
loc_df = spark.read.csv("/Volumes/workspace/hrd/rawfiles/Location.csv", header=True, inferSchema=True)
grade_df = spark.read.csv("/Volumes/workspace/hrd/rawfiles/Grade.csv", header=True, inferSchema=True)
dept_df = spark.read.csv("/Volumes/workspace/hrd/rawfiles/Dept.csv", header=True, inferSchema=True)
ass_df = spark.read.csv('/Volumes/workspace/hrd/rawfiles/Associate*.csv', header=True, inferSchema=True)

## Get Current Month data

In [0]:

from datetime import datetime
ass_df=ass_df.filter((ass_df['year'] == datetime.now().strftime("%Y")) & (ass_df['month'] == '06'))

ass_df = ass_df.join(proj_df, ass_df.projectid == proj_df.projectid, 'inner').drop(proj_df.projectid).join(loc_df, ass_df.locationid == loc_df.locationid, 'inner').drop(loc_df.locationid).join(dept_df, ass_df.departmentcode == dept_df.deptcode, 'inner').drop(dept_df.deptcode).join(grade_df, ass_df.gradecode == grade_df.gradecode, 'inner').drop(grade_df.gradecode)

##Start & End of Month values


In [0]:

from pyspark.sql.functions import trunc, current_date,last_day
from pyspark.sql.types import DateType

# Create a DataFrame with current_date
df = spark.range(1).select(trunc(current_date(), "MM").alias("start_of_month"))

# Collect the literal value
start_date_literal = df.collect()[0]["start_of_month"]

print(start_date_literal)  # Output: e.g., 2025-07-01
df = spark.range(1).select(last_day(current_date()).alias("end_of_month"))

# Collect the literal value
end_date_literal = df.collect()[0]["end_of_month"]

print(end_date_literal)


In [0]:
from pyspark.sql.functions import when, col, lit, trim,to_date
ass_df_att =ass_df.withColumn(
    "resignationdate",
    when(
        (col("resignationdate").isNull()) |
        (trim(col("resignationdate")) == "") |
        (trim(col("resignationdate")).isin("NA", "null", "NULL","None")),
        lit("0001-01-01")
    ).otherwise(col("resignationdate"))).withColumn(
    "resignationdate",
    to_date(col("resignationdate"), "yyyy-MM-dd"))
display(ass_df_att)


In [0]:
ass_df_att = ass_df_att.filter(col("resignationdate")!='0001-01-01').filter(col("resignationdate") <= end_date_literal).filter(col("resignationdate") >= start_date_literal)
display(ass_df_att)


#Write data processed files

In [0]:
ass_df.write.mode("overwrite").csv("/Volumes/workspace/hrd/processedfiles/Associates.csv")
ass_df.write.mode("overwrite").csv("/Volumes/workspace/hrd/processedfiles/Resigned_Associates.csv")